In [10]:
# import tensorflow as tf
import numpy as np
import os

def import_from_csv(path, pixel_depth):
    train_database = np.genfromtxt('{}'.format(path), delimiter=",", dtype=int)[1:,:]
    train_labels = train_database[:,0].reshape(42000,)
    training_labels = np.eye(10)[train_labels]
    training_data = np.delete(train_database,0,1)
    return normalize(training_data, pixel_depth), training_labels, train_labels

def normalize(image_data, pixel_depth):
    data = (image_data - pixel_depth / 2) / pixel_depth
    return data.astype(np.float32)

#training_data = np.random.randn(20000, 784)

#training_weights = np.random.randn(10, 784)
#training_response = np.dot(training_data, training_weights.T)

#correct_index = np.random.choice(10, np.shape(training_data)[0])
#training_response = np.eye(10)[correct_index]

training_data, training_response, _ = import_from_csv('/Users/JAustin/Desktop/MNIST/train.csv', 255)

data_size = np.shape(training_data)[0] # 20000
num_params = np.shape(training_data)[1] # 200
num_classes = np.shape(training_response)[1] # 10

weights = np.random.randn(num_classes, num_params) # or randn
biases = np.random.randn(num_classes, 1)

#learning_rate = .1 * np.ones(iterations) # constant learning rate

In [38]:
batch_size = 16
iterations = 400

# learning_rate = .005*np.exp(-5*np.arange(0,iterations)/iterations) # exponential learning rate
# learning_rate = np.array([.05*(1 - x/iterations) for x in range(iterations)]) # linear learning rate


def predict(data, batch_size, weights, biases):
    index = np.random.choice(len(data), batch_size, replace=False)
    prediction = np.dot(data[index], weights.T) + biases.T
    return index, prediction

def least_square_optimization(data, response, weights, biases, batch_size, iterations, learning_rate):
    for i in range(iterations):
        index, prediction = predict(data, batch_size, weights, biases)
        error_arr = response[index] - prediction
        loss = np.tensordot(error_arr, error_arr) / (2*batch_size)
        print("Loss at step %s is %s" % (i, loss))
        dW = np.zeros_like(weights)
        dB = np.zeros_like(biases)
        for j in range(np.shape(weights)[0]):
            for k in range(np.shape(weights)[1]):
                dW[j,k]= - np.sum((response[index]-prediction)[:, j] * data[index][:, k]) / batch_size
        for j in range(len(biases)):
            dB[j]= - np.sum((response[index]-prediction)[:, j]) / batch_size

        weights = weights - learning_rate[i]*dW
        biases = biases - learning_rate[i]*dB

    return weights, biases

def evaluate(data, labels, weights, biases, batch_size):
    index = np.random.choice(len(data), batch_size, replace=False)
    accuracy = (np.argmax(np.dot(data[index], weights.T) + biases.T, axis=1) == np.argmax(training_response[index], axis=1)).sum() * 100 / batch_size
    print("The accuracy of your model is %s%%!" % accuracy)
    return accuracy, index

In [39]:
new_weights, new_biases = least_square_optimization(training_data, training_response, weights, biases, batch_size, iterations, learning_rate)
accuracy, index = evaluate(training_data, training_response, new_weights, new_biases, 512)

Loss at step 0 is 795.232690104
Loss at step 1 is 369.590283871
Loss at step 2 is 315.500165297
Loss at step 3 is 244.897484557
Loss at step 4 is 266.580972277
Loss at step 5 is 222.600351867
Loss at step 6 is 230.246881797
Loss at step 7 is 200.248824676
Loss at step 8 is 272.850452311
Loss at step 9 is 221.820196572
Loss at step 10 is 202.023717393
Loss at step 11 is 196.336955429
Loss at step 12 is 223.617156578
Loss at step 13 is 200.736334036
Loss at step 14 is 220.798736695
Loss at step 15 is 207.211813421
Loss at step 16 is 199.203246118
Loss at step 17 is 196.612041976
Loss at step 18 is 193.850696833
Loss at step 19 is 163.962296896
Loss at step 20 is 175.101777475
Loss at step 21 is 146.810590805
Loss at step 22 is 167.125801073
Loss at step 23 is 121.059693078
Loss at step 24 is 182.25534537
Loss at step 25 is 165.228282042
Loss at step 26 is 175.598057212
Loss at step 27 is 152.473524408
Loss at step 28 is 140.334536051
Loss at step 29 is 174.103117244
Loss at step 30 is 17

In [11]:
print(np.shape(training_response))

(42000, 10)


for .05 step size, linear learning rate, loss of 88 at step 100, loss of 61 at step 200, 53 at step 400